In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [54]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import RFE, SelectFromModel

In [3]:
data = load_breast_cancer()

In [4]:
type(data)

sklearn.utils.Bunch

In [7]:
data.feature_names

array(['mean radius', 'mean texture', 'mean perimeter', 'mean area',
       'mean smoothness', 'mean compactness', 'mean concavity',
       'mean concave points', 'mean symmetry', 'mean fractal dimension',
       'radius error', 'texture error', 'perimeter error', 'area error',
       'smoothness error', 'compactness error', 'concavity error',
       'concave points error', 'symmetry error',
       'fractal dimension error', 'worst radius', 'worst texture',
       'worst perimeter', 'worst area', 'worst smoothness',
       'worst compactness', 'worst concavity', 'worst concave points',
       'worst symmetry', 'worst fractal dimension'], dtype='<U23')

In [8]:
data.target_names

array(['malignant', 'benign'], dtype='<U9')

In [11]:
data.keys()

dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names', 'filename'])

In [14]:
print(data.DESCR)

.. _breast_cancer_dataset:

Breast cancer wisconsin (diagnostic) dataset
--------------------------------------------

**Data Set Characteristics:**

    :Number of Instances: 569

    :Number of Attributes: 30 numeric, predictive attributes and the class

    :Attribute Information:
        - radius (mean of distances from center to points on the perimeter)
        - texture (standard deviation of gray-scale values)
        - perimeter
        - area
        - smoothness (local variation in radius lengths)
        - compactness (perimeter^2 / area - 1.0)
        - concavity (severity of concave portions of the contour)
        - concave points (number of concave portions of the contour)
        - symmetry 
        - fractal dimension ("coastline approximation" - 1)

        The mean, standard error, and "worst" or largest (mean of the three
        largest values) of these features were computed for each image,
        resulting in 30 features.  For instance, field 3 is Mean Radius, f

In [15]:
X = pd.DataFrame(data.data, columns = data.feature_names)

In [17]:
X.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,radius error,texture error,perimeter error,area error,smoothness error,compactness error,concavity error,concave points error,symmetry error,fractal dimension error,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [18]:
X.shape

(569, 30)

In [19]:
Y = data.target

In [20]:
Y.shape

(569,)

In [21]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [22]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((455, 30), (114, 30), (455,), (114,))

**Feature selection by feature importance of random forest classifier**

In [24]:
sel = SelectFromModel(RandomForestClassifier(n_estimators=100,n_jobs=-1,random_state=0))

In [25]:
sel.fit(x_train, y_train)

SelectFromModel(estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                                 class_weight=None,
                                                 criterion='gini',
                                                 max_depth=None,
                                                 max_features='auto',
                                                 max_leaf_nodes=None,
                                                 max_samples=None,
                                                 min_impurity_decrease=0.0,
                                                 min_impurity_split=None,
                                                 min_samples_leaf=1,
                                                 min_samples_split=2,
                                                 min_weight_fraction_leaf=0.0,
                                                 n_estimators=100, n_jobs=-1,
                                                 oob_score=False,
 

In [26]:
sel.get_support()

array([ True, False,  True,  True, False, False,  True,  True, False,
       False, False, False, False,  True, False, False, False, False,
       False, False,  True, False,  True,  True, False, False, False,
        True, False, False])

In [27]:
len(sel.get_support())

30

In [30]:
features = x_train.columns[sel.get_support()]

In [31]:
features

Index(['mean radius', 'mean perimeter', 'mean area', 'mean concavity',
       'mean concave points', 'area error', 'worst radius', 'worst perimeter',
       'worst area', 'worst concave points'],
      dtype='object')

In [32]:
len(features)

10

In [33]:
np.mean(sel.estimator_.feature_importances_)

0.03333333333333334

In [35]:
sel.estimator_.feature_importances_ # valus greater than mean are selected. Higher values means more important

array([0.03699612, 0.01561296, 0.06016409, 0.0371452 , 0.0063401 ,
       0.00965994, 0.0798662 , 0.08669071, 0.00474992, 0.00417092,
       0.02407355, 0.00548033, 0.01254423, 0.03880038, 0.00379521,
       0.00435162, 0.00452503, 0.00556905, 0.00610635, 0.00528878,
       0.09556258, 0.01859305, 0.17205401, 0.05065305, 0.00943096,
       0.01565491, 0.02443166, 0.14202709, 0.00964898, 0.01001304])

In [36]:
x_train_rfc = sel.transform(x_train)
x_test_rfc = sel.transform(x_test)

In [37]:
x_train_rfc.shape, x_test_rfc.shape

((455, 10), (114, 10))

In [38]:
def run_random_forest(x_train, x_test, y_train, y_test):
  clf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=0)
  clf.fit(x_train, y_train)
  y_pred = clf.predict(x_test)
  print('Accuracy on test set: ', accuracy_score(y_test, y_pred))

In [39]:
%%time
run_random_forest(x_train_rfc, x_test_rfc, y_train, y_test)

Accuracy on test set:  0.9473684210526315
CPU times: user 285 ms, sys: 44.8 ms, total: 330 ms
Wall time: 353 ms


In [40]:
%%time
run_random_forest(x_train, x_test, y_train, y_test)

Accuracy on test set:  0.9649122807017544
CPU times: user 366 ms, sys: 29.7 ms, total: 395 ms
Wall time: 462 ms


**Recursive Feature elimination**

In [41]:
sel = RFE(RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=0), n_features_to_select=15)

In [42]:
sel.fit(x_train, y_train)

RFE(estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                     class_weight=None, criterion='gini',
                                     max_depth=None, max_features='auto',
                                     max_leaf_nodes=None, max_samples=None,
                                     min_impurity_decrease=0.0,
                                     min_impurity_split=None,
                                     min_samples_leaf=1, min_samples_split=2,
                                     min_weight_fraction_leaf=0.0,
                                     n_estimators=100, n_jobs=-1,
                                     oob_score=False, random_state=0, verbose=0,
                                     warm_start=False),
    n_features_to_select=15, step=1, verbose=0)

In [43]:
sel.get_support()

array([ True,  True,  True,  True, False, False,  True,  True, False,
       False, False, False, False,  True, False, False, False, False,
       False, False,  True,  True,  True,  True,  True, False,  True,
        True,  True, False])

In [47]:
features = x_train.columns[sel.get_support()]

In [48]:
features

Index(['mean radius', 'mean texture', 'mean perimeter', 'mean area',
       'mean concavity', 'mean concave points', 'area error', 'worst radius',
       'worst texture', 'worst perimeter', 'worst area', 'worst smoothness',
       'worst concavity', 'worst concave points', 'worst symmetry'],
      dtype='object')

In [49]:
len(features)

15

In [50]:
x_train_rfe = sel.transform(x_train)
x_test_rfe = sel.transform(x_test)

In [51]:
x_train_rfe.shape, x_test_rfe.shape

((455, 15), (114, 15))

In [52]:
%%time
run_random_forest(x_train_rfe, x_test_rfe, y_train, y_test)

Accuracy on test set:  0.9736842105263158
CPU times: user 309 ms, sys: 27.3 ms, total: 336 ms
Wall time: 455 ms


In [53]:
%%time
run_random_forest(x_train, x_test, y_train, y_test)

Accuracy on test set:  0.9649122807017544
CPU times: user 363 ms, sys: 24.5 ms, total: 387 ms
Wall time: 458 ms


**Feature selection by gradient boost algorithm/ Tree importance**

In [55]:
sel = RFE(GradientBoostingClassifier(n_estimators=100, random_state=0), n_features_to_select=12)

In [56]:
sel.fit(x_train, y_train)

RFE(estimator=GradientBoostingClassifier(ccp_alpha=0.0,
                                         criterion='friedman_mse', init=None,
                                         learning_rate=0.1, loss='deviance',
                                         max_depth=3, max_features=None,
                                         max_leaf_nodes=None,
                                         min_impurity_decrease=0.0,
                                         min_impurity_split=None,
                                         min_samples_leaf=1,
                                         min_samples_split=2,
                                         min_weight_fraction_leaf=0.0,
                                         n_estimators=100,
                                         n_iter_no_change=None,
                                         presort='deprecated', random_state=0,
                                         subsample=1.0, tol=0.0001,
                                         validation_frac

In [57]:
sel.get_support()

array([False,  True, False, False,  True, False, False,  True,  True,
       False, False, False, False,  True, False, False,  True, False,
       False, False,  True,  True,  True,  True, False, False,  True,
        True, False, False])

In [58]:
features = x_train.columns[sel.get_support()]

In [59]:
features

Index(['mean texture', 'mean smoothness', 'mean concave points',
       'mean symmetry', 'area error', 'concavity error', 'worst radius',
       'worst texture', 'worst perimeter', 'worst area', 'worst concavity',
       'worst concave points'],
      dtype='object')

In [60]:
len(features)

12

In [61]:
x_train_gbe = sel.transform(x_train)
x_test_gbe = sel.transform(x_test)

In [63]:
x_train_gbe.shape, x_test_gbe.shape

((455, 12), (114, 12))

In [64]:
%%time
run_random_forest(x_train_gbe, x_test_gbe, y_train, y_test)

Accuracy on test set:  0.9736842105263158
CPU times: user 298 ms, sys: 37.2 ms, total: 335 ms
Wall time: 361 ms


In [65]:
%%time
run_random_forest(x_train, x_test, y_train, y_test)

Accuracy on test set:  0.9649122807017544
CPU times: user 330 ms, sys: 42.6 ms, total: 373 ms
Wall time: 457 ms


In [66]:
for index in range(1,31):
  sel = RFE(GradientBoostingClassifier(n_estimators=100, random_state=0), n_features_to_select=index)
  sel.fit(x_train, y_train)
  x_train_gbe = sel.transform(x_train)
  x_test_gbe = sel.transform(x_test)
  print('Selected: ', index)
  %%time
  run_random_forest(x_train_gbe, x_test_gbe, y_train, y_test)

Selected:  1
CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.91 µs
Accuracy on test set:  0.8771929824561403
Selected:  2
CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.2 µs
Accuracy on test set:  0.9035087719298246
Selected:  3
CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 6.44 µs
Accuracy on test set:  0.9649122807017544
Selected:  4
CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.96 µs
Accuracy on test set:  0.9736842105263158
Selected:  5
CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.44 µs
Accuracy on test set:  0.9649122807017544
Selected:  6
CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.39 µs
Accuracy on test set:  0.9912280701754386
Selected:  7
CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.2 µs
Accuracy on test set:  0.9736842105263158
Selected:  8
CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.63 µs
Accuracy on test set:  0.9649122807017544
Selected:  9
CPU times: user 3 µs, sys: 1 µs, tota

In [68]:
sel = RFE(GradientBoostingClassifier(n_estimators=100, random_state=0), n_features_to_select=6)
sel.fit(x_train, y_train)
x_train_gbe = sel.transform(x_train)
x_test_gbe = sel.transform(x_test)
print('Selected: ', 6)
run_random_forest(x_train_gbe, x_test_gbe, y_train, y_test)

Selected:  6
Accuracy on test set:  0.9912280701754386


In [70]:
features = x_train.columns[sel.get_support()]

In [71]:
features

Index(['mean concave points', 'area error', 'worst texture', 'worst perimeter',
       'worst area', 'worst concave points'],
      dtype='object')

In [72]:
for index in range(1,31):
  sel = RFE(RandomForestClassifier(n_estimators=100, random_state=0), n_features_to_select=index)
  sel.fit(x_train, y_train)
  x_train_gbe = sel.transform(x_train)
  x_test_gbe = sel.transform(x_test)
  print('Selected: ', index)
  %%time
  run_random_forest(x_train_gbe, x_test_gbe, y_train, y_test)

Selected:  1
CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.91 µs
Accuracy on test set:  0.8947368421052632
Selected:  2
CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.96 µs
Accuracy on test set:  0.9298245614035088
Selected:  3
CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.72 µs
Accuracy on test set:  0.9473684210526315
Selected:  4
CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.15 µs
Accuracy on test set:  0.9649122807017544
Selected:  5
CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.44 µs
Accuracy on test set:  0.9649122807017544
Selected:  6
CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.96 µs
Accuracy on test set:  0.956140350877193
Selected:  7
CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.2 µs
Accuracy on test set:  0.956140350877193
Selected:  8
CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 6.2 µs
Accuracy on test set:  0.9649122807017544
Selected:  9
CPU times: user 3 µs, sys: 0 ns, total: